In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santo antonio do ica,-3.1022,-67.9397,25.11,62,32,1.32,BR,1724983762
1,1,waitangi,-43.9535,-176.5597,13.79,91,55,2.24,NZ,1724983605
2,2,udachny,66.4167,112.4000,12.31,49,1,2.87,RU,1724983609
3,3,port-aux-francais,-49.3500,70.2167,3.27,70,99,11.76,TF,1724983649
4,4,nouadhibou,20.9310,-17.0347,22.99,73,20,8.75,MR,1724983767
...,...,...,...,...,...,...,...,...,...,...
564,564,rozdilna,46.8433,30.0792,20.59,58,100,3.11,UA,1724984434
565,565,sakaraha,-22.9000,44.5333,15.47,83,1,0.57,MG,1724984435
566,566,mtambile,-5.3833,39.7000,24.33,80,7,6.09,TZ,1724984436
567,567,bonthe,7.5264,-12.5050,24.40,89,34,4.92,SL,1724984438


In [17]:
# Configure the map plot
latitude = 'Lat'
longitude = 'Lng'

map_plot = city_data_df.hvplot.points(
    x=longitude,
    y=latitude,
    geo=True,
    tiles="OSM",               
    frame_width=800,           
    frame_height=600,           
    size='Humidity',          
    color='Humidity',          
    alpha=0.6,                  
    hover_cols=["City"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [21]:
# Ideal weather criteria
temperature_min = 21
temperature_max = 27
wind_speed_max = 4.5
cloudiness_max = 0

# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > temperature_min) & 
    (city_data_df['Max Temp'] < temperature_max) &
    (city_data_df['Wind Speed'] < wind_speed_max) &
    (city_data_df['Cloudiness'] == cloudiness_max)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
48,48,gargalianoi,37.0667,21.6333,23.23,83,0,3.11,GR,1724983818
49,49,newnan,33.3807,-84.7997,26.10,77,0,0.00,US,1724983785
76,76,foggia,41.4500,15.5333,24.79,60,0,2.98,IT,1724983575
86,86,petropavlovsk-kamchatsky,53.0452,158.6483,21.33,46,0,4.00,RU,1724983807
87,87,la foa,-21.7108,165.8276,25.62,44,0,1.50,NC,1724983864
122,122,severo-vostotchnyi bank,39.4112,49.2479,23.85,68,0,2.74,AZ,1724983907
186,186,saveh,35.0213,50.3566,23.95,23,0,1.77,IR,1724983983
215,215,kas,36.2018,29.6377,26.27,58,0,2.04,TR,1724984017
223,223,yeppoon,-23.1333,150.7333,24.90,58,0,4.08,AU,1724984026
246,246,beyneu,45.3167,55.2000,23.14,33,0,1.61,KZ,1724984053


In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
48,gargalianoi,GR,37.0667,21.6333,83,
49,newnan,US,33.3807,-84.7997,77,
76,foggia,IT,41.4500,15.5333,60,
86,petropavlovsk-kamchatsky,RU,53.0452,158.6483,46,
87,la foa,NC,-21.7108,165.8276,44,
122,severo-vostotchnyi bank,AZ,39.4112,49.2479,68,
186,saveh,IR,35.0213,50.3566,23,
215,kas,TR,36.2018,29.6377,58,
223,yeppoon,AU,-23.1333,150.7333,58,
246,beyneu,KZ,45.3167,55.2000,33,


In [25]:
radius = 5000 
params = {
    'categories': 'accommodation.hotel',  
    'limit': 1,                          
    'apiKey': geoapify_key                
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"  

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gargalianoi - nearest hotel: Messinian Horizons
newnan - nearest hotel: Casa Bella Bed & Breakfast
foggia - nearest hotel: Hotel Bella Napoli
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
la foa - nearest hotel: No hotel found
severo-vostotchnyi bank - nearest hotel: No hotel found
saveh - nearest hotel: هتل صدرا
kas - nearest hotel: KEKOVA OTEL
yeppoon - nearest hotel: Blossom's on Seaspray
beyneu - nearest hotel: Нұр
san pedro de alcantara - nearest hotel: Hotel NH San Pedro
tortoli - nearest hotel: La Corte
riberalta - nearest hotel: Hotel Jomali
remire-montjoly - nearest hotel: Complexe Belova
santarem - nearest hotel: Prático Hotel
greencastle - nearest hotel: Inn at DePauw
partinico - nearest hotel: No hotel found
benton harbor - nearest hotel: Silver Beach Hotel
pergamos - nearest hotel: No hotel found
zhangjiakou - nearest hotel: 商务宾馆
vilhena - nearest hotel: No hotel found
box elder - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
48,gargalianoi,GR,37.0667,21.6333,83,Messinian Horizons
49,newnan,US,33.3807,-84.7997,77,Casa Bella Bed & Breakfast
76,foggia,IT,41.4500,15.5333,60,Hotel Bella Napoli
86,petropavlovsk-kamchatsky,RU,53.0452,158.6483,46,ООО Постоялый двор
87,la foa,NC,-21.7108,165.8276,44,No hotel found
122,severo-vostotchnyi bank,AZ,39.4112,49.2479,68,No hotel found
186,saveh,IR,35.0213,50.3566,23,هتل صدرا
215,kas,TR,36.2018,29.6377,58,KEKOVA OTEL
223,yeppoon,AU,-23.1333,150.7333,58,Blossom's on Seaspray
246,beyneu,KZ,45.3167,55.2000,33,Нұр


In [30]:
# Configure the map plot

map_plot2 = hotel_df.hvplot.points(
x='Lng',                   
    y='Lat',                   
    geo=True,
    tiles="OSM",               
    frame_width=800,           
    frame_height=600,          
    size='Lat',           
    scale=2,                   
    color='Humidity',                      
    alpha=0.6,                         
    hover_cols=["City","Humidity", "Hotel Name", "Country"]
)
# Display the map plot
map_plot2

/opt/anaconda3/envs/dev/lib/python3.12/site-packages/holoviews/util/transform.py:655: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)